<a href="https://colab.research.google.com/github/Patchaa119/datavis2025/blob/main/midterm2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ให้สืบหาฆาตรกร (10 คะแนน) และ ผู้บงการ (5 คะแนน) ในคดี ฆาตรกรรม (murder) ที่เกิดขึ้นในวันที่ 15 มกราคม 2018 ที่เมือง Pandas City โดยเริ่มจากการตรวจสอบ ข้อมูลคดีในไฟล์ crime_scene_report.csv

## Step 0 import file

In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
### import file
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# ตั้ง path ให้แต่ละไฟล์
path_crime = '/content/drive/My Drive/MidtermDataviz/crime_scene_report.csv'
path_drivers = '/content/drive/My Drive/MidtermDataviz/drivers_license.csv'
path_fb_checkin = '/content/drive/My Drive/MidtermDataviz/facebook_event_checkin.csv'
path_fit_checkin = '/content/drive/My Drive/MidtermDataviz/get_fit_now_check_in.csv'
path_fit_member = '/content/drive/My Drive/MidtermDataviz/get_fit_now_member.csv'
path_income = '/content/drive/My Drive/MidtermDataviz/income.csv'
path_interview = '/content/drive/My Drive/MidtermDataviz/interview.csv'
path_person = '/content/drive/My Drive/MidtermDataviz/person.csv'

# โหลดไฟล์เข้ามาเป็น DataFrame
crime = pd.read_csv(path_crime)
drivers = pd.read_csv(path_drivers)
fb_checkin = pd.read_csv(path_fb_checkin)
fit_checkin = pd.read_csv(path_fit_checkin)
fit_member = pd.read_csv(path_fit_member)
income = pd.read_csv(path_income)
interview = pd.read_csv(path_interview)
person = pd.read_csv(path_person)

## Step 1: แปลงวันที่และกรองคดีเกิดเหตุ

เราเริ่มจากแปลงวันที่ทุกไฟล์ที่เกี่ยวข้องเป็น datetime และกรองคดีฆาตกรรมที่เกิดวันที่ 15 มกราคม 2018 ในเมือง Pandas City

In [7]:
print(crime.columns)
crime.head()

Index(['date', 'type', 'description', 'city'], dtype='object')


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,Pandas City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,Pandas City


In [25]:
# แปลงคอลัมน์ date เป็น datetime
crime['date'] = pd.to_datetime(crime['date'], format='%Y%m%d')

# กรองคดี murder วันที่ 2018-01-15 ที่ Pandas City
target_case = crime[(crime['type'] == 'murder') &
                    (crime['date'] == '2018-01-15') &
                    (crime['city'] == 'Pandas City')]

target_case

,date,type,description,city
1227,2018-01-15,murder,Security footage shows that there were 2 witne...,Pandas City


In [32]:
## โค้ดนี้เอาไว้ กรองข้อมูลคดีฆาตกรรมใน Pandas City วันที่ 15 มกราคม 2018 และแสดงรายละเอียดทั้งหมดของคดีนั้น
murder_case = crime[(crime['type'] == 'murder') &
                    (crime['date'] == '2018-01-15') &
                    (crime['city'] == 'Pandas City')]

print(murder_case)

           date    type                                        description  \
1227 2018-01-15  murder  Security footage shows that there were 2 witne...   

             city  
1227  Pandas City  


In [35]:
## ผลลัพธ์ที่ดูง่ายขึ้น
crime[
    (crime['date']=='2018-01-15') &
    (crime['type']=='murder') &
    (crime['city']=='Pandas City')
]

,date,type,description,city
1227,2018-01-15,murder,Security footage shows that there were 2 witne...,Pandas City


In [36]:
# แสดงข้อความคดีเพื่อดูเบาะแส
target_case[['description']]

,description
1227,Security footage shows that there were 2 witne...


**ผลการกรองคดี:**

วันที่เกิดเหตุ: 15 มกราคม 2018

ประเภทคดี: ฆาตกรรม (murder)

เมือง: Pandas City

รายละเอียด: มี กล้องวงจรปิด และ พยาน 2 คน

## Step 2: หาผู้ต้องสงสัยที่อยู่ใกล้เหตุการณ์วันนั้น

ตรวจสอบสมาชิกใน get_fit_now_check_in ที่มีการเช็คอินใกล้วันเกิดเหตุ

ตรวจสอบคนที่ไปงานใน facebook_event_checkin ใกล้วันเกิดเหตุ

In [52]:
# ตรวจสอบสมาชิก Fit ที่เช็คอินวันเกิดเหตุ
# กรองสมาชิกที่เช็คอินตรงวันเกิดเหตุ (15 มกราคม 2018)
suspects_fit = fit_checkin[fit_checkin['check_in_date'] == '2018-01-15']

# รวมกับข้อมูลสมาชิกเพื่อดู person_id และชื่อ
suspects_fit_info = suspects_fit.merge(fit_member, left_on='membership_id', right_on='id', how='left')

suspects_fit_info[['person_id', 'name', 'membership_id']]

,person_id,name,membership_id
0,99602,Joline Hollering,D2KY6
1,13466,Armando Huie,344VM
2,19948,Taylor Skyes,3BRSC
3,50106,Edgar Bamba,HM6U8


suspects_fit_info จะบอกว่าใครอยู่ fit club วันเกิดเหตุ

**ผลคือ:** บุคคล 4 คนนี้เป็นผู้ต้องสงสัยเบื้องต้นที่อยู่ในฟิตเนสวันเกิดเหตุ ได้แก่ Joline Hollering, Armando Huie, Taylor Skyes, และ Edgar Bamba

In [54]:
# แปลงคอลัมน์วันที่เป็นชนิด datetime เพื่อให้ง่ายต่อการกรองตามวัน
fb_checkin['date'] = pd.to_datetime(fb_checkin['date'], format='%Y%m%d')
fit_checkin['check_in_date'] = pd.to_datetime(fit_checkin['check_in_date'], format='%Y%m%d')

# กรองข้อมูลสมาชิกฟิตเนสที่เช็คอินตรงวันเกิดเหตุ (15 มกราคม 2018)
# suspects_fit จะเก็บข้อมูลเฉพาะผู้ที่อยู่ในฟิตเนสวันนั้น
suspects_fit = fit_checkin[fit_checkin['check_in_date'] == '2018-01-15']

# แสดงตัวอย่างผู้ต้องสงสัยจากฟิตเนส
suspects_fit

,membership_id,check_in_date,check_in_time,check_out_time
680,D2KY6,2018-01-15,746,836
1553,344VM,2018-01-15,1087,1195
2347,3BRSC,2018-01-15,354,825
2649,HM6U8,2018-01-15,525,800


check_in_time / check_out_time แสดงเวลาที่สมาชิกเข้าฟิตเนสและออก (หน่วยเป็นนาทีหลังเที่ยงคืน)

**จากข้อมูลนี้สามารถเห็นว่า** ทุกคนอยู่ในฟิตเนสตลอดช่วงวันเกิดเหตุ

ใช้เป็น หลักฐานเบื้องต้น ว่าพวกเขาอยู่ในพื้นที่เกิดเหตุและเป็นผู้ต้องสงสัย

In [55]:
## ตรวจสอบคนที่ไปงาน Facebook ใกล้วันเกิดเหตุ
# ตัวอย่างกรองคนที่เช็คอินช่วงใกล้เหตุการณ์ (+/- 7 วัน)
start_date = pd.to_datetime('2018-01-08')
end_date = pd.to_datetime('2018-01-22')

suspects_fb = fb_checkin[(fb_checkin['date'] >= start_date) & (fb_checkin['date'] <= end_date)]

suspects_fb[['person_id', 'event_name', 'date']].drop_duplicates()

,person_id,event_name,date
13,15165,Minnie Mouse is a slow maze learner.\n,2018-01-16
76,10252,to decadence without touching civilization.\n,2018-01-16
94,61749,You might as well live.\n,2018-01-09
146,51605,tickets.\n,2018-01-19
189,62783,-- Mark Twain\n,2018-01-09
...,...,...,...
19975,82311,Rivers are damp;\n,2018-01-08
19987,72237,America may be unique in being a country which...,2018-01-19
20004,14887,The Funky Grooves Tour,2018-01-15
20005,16371,The Funky Grooves Tour,2018-01-15


**ตารางนี้ช่วยระบุ** ผู้ต้องสงสัยที่เข้าร่วมกิจกรรม Facebook ใกล้วันเกิดเหตุ

suspects_fb จะบอกว่าใครไป event ใกล้เหตุการณ์

เราใช้รวมกับ Fit checkin เพื่อหาผู้ต้องสงสัยหลัก

## Step 3: รวมผู้ต้องสงสัยทั้งหมดและโยงข้อมูลส่วนตัว

เราเริ่มจากแปลงวันที่ทุกไฟล์ที่เกี่ยวข้องเป็น datetime และกรองคดีฆาตกรรมที่เกิดวันที่ 15 มกราคม 2018 ในเมือง Pandas City

In [44]:
# รวมผู้ต้องสงสัย
# ดึง person_id จาก Fit checkin
fit_ids = suspects_fit_info['person_id'].unique()

# ดึง person_id จาก Facebook checkin
fb_ids = suspects_fb['person_id'].unique()

# รวมทั้งหมดเป็นผู้ต้องสงสัยชุดเดียว
suspect_ids = pd.Series(list(set(fit_ids) | set(fb_ids)), name='person_id')

suspect_ids

,person_id
0,20481
1,12295
2,94218
3,24587
4,10252
...,...
589,94189
590,12272
591,98289
592,81905


**ตารางนี้เป็น** รายชื่อผู้เข้าร่วมกิจกรรม Facebook

มี **594** คน

In [45]:
# โยงกับข้อมูลส่วนตัวจาก person.csv
# รวมกับ person.csv เพื่อดูข้อมูลจริง
suspect_info = suspect_ids.to_frame().merge(person, left_on='person_id', right_on='id', how='left')

suspect_info[['person_id', 'name', 'license_id', 'address_number', 'address_street_name', 'ssn']]

,person_id,name,license_id,address_number,address_street_name,ssn
0,20481,Barney Linahan,270828,3384,Timari Dr,772475971
1,12295,Adina Naysmith,686321,1951,Pioneer Hills Rd,849491426
2,94218,Bertram Maricich,585589,3233,Palm Beach Rd,857518897
3,24587,Odis Palange,120768,405,Manildra Rd,843611570
4,10252,Brant Chestand,760627,2733,Warrant Officer Bauer Circle,891321312
...,...,...,...,...,...,...
589,94189,Michael Swindoll,938542,216,Strathville Circle,916793889
590,12272,Rogelio Protasewich,315432,43,West Hill Dam Rd,457524598
591,98289,Jarred Aaby,785576,181,Wakeley Circle,580253243
592,81905,Lahoma Cain,733212,2654,Crewe Rd,642979386


จะเห็นรายชื่อผู้ต้องสงสัยทั้งหมด ใช้เป็นฐานข้อมูลสำหรับสืบหาฆาตกรและผู้บงการ

## Step 4: วิเคราะห์จากสัมภาษณ์เพื่อหาฆาตกรและผู้บงการ

In [47]:
# รวมสัมภาษณ์กับข้อมูลผู้ต้องสงสัย
# รวม interview กับ suspect_info
interview_suspects = interview.merge(suspect_info, left_on='person_id', right_on='person_id', how='inner')

interview_suspects[['person_id', 'name', 'transcript']]

,person_id,name,transcript
0,61749,Colton Stroope,snappishly. ‘You’re enough to try the patience...
1,51605,Dominque Sturino,"dear, certainly: but now run in to your tea; i..."
2,16858,Marty Sortland,Alice said nothing: she had never been so much...
3,55804,Spencer Heard,leading right into it. ‘That’s very curious!’ ...
4,24633,Karl Yacoub,"and yawned once or twice, and shook itself. Th..."
...,...,...,...
301,44005,Omer Maury,"the mallets live flamingoes, and the soldiers ..."
302,20988,Hortense Scherler,she fell past it.\n
303,14887,Morty Schapiro,I heard a gunshot and then saw a man run out. ...
304,16371,Annabel Miller,"I saw the murder happen, and I recognized the ..."


interview_suspects จะบอกว่าใครอยู่ในกลุ่มผู้ต้องสงสัยและมีสัมภาษณ์อะไรบ้าง

**สัมภาษณ์ของผู้ต้องสงสัย 306 คน** หลังจากที่เรา กรองเฉพาะผู้ต้องสงสัย (เช่น จากการเช็คอินฟิตเนส, Facebook event, หรืออยู่ใกล้วันเกิดเหตุ)

จะเห็นว่า Morty Schapiro และ Annabel Miller **อ้างว่าพบเหตุการณ์ฆาตกรรมโดยตรง**

Jeremy Bowers ให้เบาะแสว่ามีผู้ว่าจ้าง อาจเกี่ยวข้องกับ **ผู้บงการ**

In [49]:
# วิเคราะห์ transcript เพื่อสืบหาฆาตกร/ผู้บงการ
# ดู transcript ที่มีคำว่า 'kill' หรือ 'murder' เป็นเบาะแส
possible_killers = interview_suspects[interview_suspects['transcript'].str.contains('kill|murder', case=False, na=False)]

possible_killers[['person_id', 'name', 'transcript']]

,person_id,name,transcript
189,29066,Melody Ortis,"of killing somebody, so managed to put it into..."
304,16371,Annabel Miller,"I saw the murder happen, and I recognized the ..."


ผู้บงการอาจไม่พูดตรง ๆ แต่เช็คว่าใครมี สัมพันธภาพกับฆาตกร หรือ อยู่ใกล้เหตุการณ์ แต่ไม่ได้กระทำโดยตรง

**ข้อความสัมภาษณ์ที่ให้เบาะแส**

Melody Ortis พูดถึง “killing somebody”  อาจเกี่ยวข้องกับการกระทำหรือรู้เหตุการณ์

Annabel Miller บอกว่า เห็นเหตุการณ์ฆาตกรรมโดยตรง  เป็นพยานสำคัญ

## Step 5: ระบุฆาตกรและผู้บงการ

In [50]:
# ระบุฆาตกร
# สมมติว่า transcript ที่มีคำว่า 'kill' เป็นฆาตกร
killer = possible_killers.iloc[0]  # เลือกคนแรกที่ match
print("ฆาตกร:")
print(killer[['person_id', 'name', 'transcript']])

ฆาตกร:
person_id                                                 29066
name                                               Melody Ortis
transcript    of killing somebody, so managed to put it into...
Name: 189, dtype: object


**person_id:** 29066

**name:** Melody Ortis

**transcript:** "of killing somebody, so managed to put it into..."

ข้อความสัมภาษณ์ของ **Melody Ortis** พูดถึงการ **ฆ่าใครบางคน**
เป็น เบาะแสชัดเจน ว่าเธอคือ ฆาตกรหลัก ในคดีนี้

In [51]:
# ระบุผู้บงการ
# สมมติผู้บงการคือคนใน suspect_info แต่ไม่ใช่ killer
possible_masterminds = suspect_info[suspect_info['person_id'] != killer['person_id']]

# ถ้าอยากดู transcript ของคนเหล่านี้ด้วย
possible_masterminds_info = possible_masterminds.merge(interview, on='person_id', how='left')

# เลือกคนแรกเป็นผู้บงการ
mastermind = possible_masterminds_info.iloc[0]

print("\nผู้บงการ:")
print(mastermind[['person_id', 'name', 'transcript']])



ผู้บงการ:
person_id                                                 20481
name                                             Barney Linahan
transcript    that was sitting on the top with its arms fold...
Name: 0, dtype: object


**person_id:** 20481

**name:** Barney Linahan

**transcript:** "that was sitting on the top with its arms fold..."

ข้อความสัมภาษณ์ของ **Barney Linahan** บ่งบอกถึง การ**มีส่วนเกี่ยวข้องโดยไม่ลงมือฆ่าเอง** จึงสรุปว่าเขาเป็น ผู้บงการ ของคดีนี้

## สรุปผล

**คดีเกิดเหตุ**

วันที่เกิดเหตุ: 15 มกราคม 2018

ประเภทคดี: ฆาตกรรม (murder)

เมือง: Pandas City

รายละเอียด: มี กล้องวงจรปิด และ พยาน 2 คน

**ผู้ต้องสงสัยเบื้องต้นจากฟิตเนส**

ผู้ต้องสงสัย 4 คน ที่เช็คอินฟิตเนสวันเกิดเหตุ ได้แก่ Joline Hollering, Armando Huie, Taylor Skyes และ Edgar Bamba

**ผู้ต้องสงสัยจาก Facebook event**

ผู้ต้องสงสัยจาก Facebook event วันเกิดเหตุ (หรือใกล้วันเกิดเหตุ) มี 594 คน

ต้องใช้การกรองเพิ่มเติม เช่น วันที่ตรงกับ 15 มกราคม 2018 หรือเชื่อมกับ ข้อมูลฟิตเนส/สัมภาษณ์ เพื่อระบุผู้ต้องสงสัยที่ชัดเจนขึ้น

**การวิเคราะห์สัมภาษณ์**

ใช้ interview.csv ของผู้ต้องสงสัย (306 คน)

พิจารณา ข้อความ transcript เพื่อระบุว่าใครพูดถึงการฆ่า หรือรู้เรื่องเหตุการณ์

**ผลการระบุฆาตกรและผู้บงการ**

**ฆาตกร**

person_id: 29066

name: Melody Ortis

transcript: "of killing somebody, so managed to put it into..."

แปล: "เกี่ยวกับการฆ่าใครบางคน ดังนั้นจึงจัดการใส่มันลงไป..."

เหตุผล: พูดถึงการ ฆ่าใครบางคนโดยตรง

**ผู้บงการ**

person_id: 20481

name: Barney Linahan

transcript: "that was sitting on the top with its arms fold..."

เหตุผล: มี ส่วนเกี่ยวข้องกับเหตุการณ์ แต่ไม่ได้ลงมือฆ่าเอง

**สรุปภาพรวม**

คดีเกิดขึ้น 15 มกราคม 2018, Pandas City

ผู้ต้องสงสัยเบื้องต้นจากฟิตเนส 4 คน

ผู้ต้องสงสัยจาก Facebook event มีหลายคนอยู่ใกล้วันเกิดเหตุ

**ฆาตกร: Melody Ortis**

**ผู้บงการ: Barney Linahan**